In [6]:
# import libraries
import os
import json
from openai import OpenAI
import gradio as gr

In [51]:
# function for building prompt


def generate_prompt(input_snippet):
    example_inputSnippet = """
    def calculate_sum(a, b):
    return a + b
    """

    example_output = '''
    {
    "language": "python"
    "code":"
    def calculate_sum(a: float, b: float) -> float:
    """
    Calculate the sum of two numeric values.

    Parameters:
        a (float): First number to be added.
        b (float): Second number to be added.

    Returns:
        float: The result of adding a and b.
    """

    # Perform addition operation
    result = a + b

    # Return the computed sum
    return result
    "
    }
    '''

    system_prompt = f"""
    You are a code comment and docstring generator.
    You add helpful comment and docstring to a given code snippet

    For example, give a python snippet {example_inputSnippet}, you will respond with {example_output}
    """

    user_prompt = f"""
    Here is a code snippet input: {input_snippet}

    Task
    - Add helpful comment and/or docstring using the syntax of the programming language detected

    Rules:
    - Response should be a valid JSON 
    - the value of "code" property of the JSON must be be a valid codesnippet in detected language
    - No Explanation.
    - Do not wrap response in code block
    """

    return system_prompt, user_prompt

In [45]:
# using models available via openrouter

OPENROUTER_URL = "https://openrouter.ai/api/v1"
openrouter_key = os.getenv('OPENROUTER_API_KEY')
openrouter = OpenAI(
        base_url= OPENROUTER_URL,
        api_key= openrouter_key
    )

def openrouter_model(model, system_prompt, user_prompt):
    
    messages = [
        { "role": "system", "content": system_prompt },
        { "role": "user", "content": user_prompt },
    ]

    response  = openrouter.chat.completions.create(
        model = model,
        messages = messages,
         max_tokens=700
    )

    output = response.choices[0].message.content.strip()

    return output

In [47]:
#map model to provider
model_to_provider = {
    'openai/gpt-5.3-codex': 'openrouter',
    'anthropic/claude-sonnet-4.6': 'openrouter',
}

MODEL_GPT = 'anthropic/claude-sonnet-4.6'

def get_model_provider(model=MODEL_GPT):
    return model_to_provider.get(model, 'openrouter')


In [39]:
#let's route request to provider based on model

def route_to_provider (model, system_prompt, user_prompt): 
    provider = get_model_provider(model)
    
    if provider == 'openrouter':
        return openrouter_model(model, system_prompt, user_prompt)
    else:
        # handle unknown provider
        response = f"No provider is implemented for the {model} yet"
        print(response)
        return response  

In [40]:
# define test code callback function  for Gradio

def generate_code_comment (model, snippet):
    
    system_prompt, user_prompt = generate_prompt(snippet)

    return route_to_provider(model, system_prompt, user_prompt) 

In [49]:

def validate_output(output):
    print(output)
    try:
        return json.loads(output)
    except json.JSONDecodeError:
        return False

def display_code_output(output):
  responseJson = validate_output(output) or {"code": 'output', "language": 'python'}
  return gr.update(value=responseJson['code'], language=responseJson['language'])

In [ ]:



with gr.Blocks() as ui:
  gr.Markdown("## Code Helper Assistant")
  with gr.Row():
    model_selector = gr.Radio(label="Select Model", choices=list(model_to_provider.keys()), value=MODEL_GPT)
  with gr.Row(equal_height=True):
    with gr.Column():
      code_input = gr.TextArea(label="Enter your code snippet here", lines=10)
    with gr.Column():
      output = gr.Code(label="Output", lines=20)
  with gr.Row():
    generate_docstring_btn = gr.Button("Generate Docstrings")

  generate_docstring_btn.click(
    lambda code, model: display_code_output(generate_code_comment(model, code)),
    inputs=[code_input, model_selector],
    outputs=output
  )


ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7888
* To create a public link, set `share=True` in `launch()`.


{
  "language": "python",
  "code": "import json\nfrom typing import Any, Optional\n\ndef parse_json(value: str) -> Optional[Any]:\n    \"\"\"\n    Parse a JSON-formatted string and return the corresponding Python object.\n\n    Parameters:\n        value (str): A JSON-formatted string to be parsed.\n\n    Returns:\n        Optional[Any]: The Python object represented by the JSON string,\n                       or None if the string is not valid JSON.\n    \"\"\"\n    try:\n        # Attempt to deserialize the JSON string into a Python object\n        return json.loads(value)\n    except json.JSONDecodeError:\n        # Return None if the string is not a valid JSON format\n        return None\n"
}
{
  "language": "python",
  "code": "import json\nfrom typing import Any, Optional\n\n\ndef parse_json(value: str) -> Optional[Any]:\n    \"\"\"\n    Parse a JSON-formatted string into a Python object.\n\n    Args:\n        value (str): A string expected to contain valid JSON.\n\n    Returns: